In [4]:
! pip install -U sentence-transformers 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 7.9 MB/s eta 0:00:00


In [5]:
import torch
from transformers import AutoTokenizer
from torch.utils.data import DataLoader, Dataset
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import json
from sklearn.preprocessing import LabelEncoder
import pandas as pd


class ContractNLIDataset(Dataset):
    def __init__(self, premises, hypotheses, labels, embedder):
        self.premises = premises
        self.hypotheses = hypotheses
        self.labels = labels
        self.embedder = embedder
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        # Concatenate premise and hypothesis for embedding
        text = f"{self.premises[idx]} [SEP] {self.hypotheses[idx]}"
        embedding = self.embedder.encode(text)
        return {
            'embedding': torch.tensor(embedding, dtype=torch.float32),
            'label': self.labels[idx]
        }

def prepare_jina_embedding_model():
    # Use Jina long context embedding model
    embedder = SentenceTransformer('jinaai/jina-embeddings-v2-small-en')
    return embedder

def train_classifier(train_embeddings, train_labels, test_embeddings, test_labels):
    # Train Logistic Regression classifier
    clf = LogisticRegression(max_iter=1000)
    clf.fit(train_embeddings, train_labels)
    
    # Predict and evaluate
    train_pred = clf.predict(train_embeddings)
    test_pred = clf.predict(test_embeddings)
    
    print("Train Accuracy:", accuracy_score(train_labels, train_pred))
    print("Test Accuracy:", accuracy_score(test_labels, test_pred))
    
    return clf

def prepare_data(json_path):
    # Read JSON file
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    # Convert JSON to DataFrame
    df = pd.DataFrame(data)
    
    # Convert verdicts to numerical labels
    label_encoder = LabelEncoder()
    labels = label_encoder.fit_transform(df['label'])
    
    # Select texts, hypotheses, and labels for the specified dataset type
    texts = df['document'].values
    hypotheses = df['hypothesis'].values
    
    return (texts, hypotheses, labels), label_encoder

In [6]:
(train_texts, train_hypotheses, train_labels), \
label_encoder = prepare_data('/kaggle/input/anlp-dataset/train.json')

In [7]:
embedder = prepare_jina_embedding_model()

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/70.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
train_dataset = ContractNLIDataset(train_texts, train_hypotheses, train_labels, embedder)

from tqdm import tqdm

# Prepare embeddings with tqdm progress bar
embeddings = []
labels = []

# Use tqdm to iterate over the dataset and show progress
for item in tqdm(train_dataset, desc="Generating Embeddings"):
    embeddings.append(item['embedding'].numpy())
    labels.append(item['label'])

# Convert lists to numpy arrays
embeddings = torch.tensor(embeddings, dtype=torch.float32).numpy()
labels = torch.tensor(labels, dtype=torch.int64).numpy()

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    embeddings, labels, test_size=0.2, random_state=42
)

Generating Embeddings:   0%|          | 0/7191 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.
Generating Embeddings:   0%|          | 1/7191 [00:00<48:33,  2.47it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Embeddings:  99%|█████████▉| 7140/7191 [07:38<00:05,  8.54it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Embeddings:  99%|█████████▉| 7141/7191 [07:39<00:09,  5.53it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
train_classifier(X_train, y_train, X_test, y_test)

Train Accuracy: 0.6220445062586927
Test Accuracy: 0.5990271021542738


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000)